In [34]:
import os, torch, copy, sys, random,cv2
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

In [35]:
RANDOM_SEED = 1001

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [36]:
# parameters

### 데이터 디렉토리 설정 ###
WORKING_DIR = '/USER/Taeyun/'
NUM_CLS = 1

EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']
    def append(self,img,label):
        pass
        


In [9]:
model_Alex = models.AlexNet(num_classes=1)
model_Alex.features[0] = nn.Conv2d(1,64,kernel_size=(11,11),stride=(4,4),padding=(2,2))
model_Alex.cuda()

RuntimeError: CUDA error: out of memory

In [18]:
class custom_CNN(nn.Module):
    def __init__(self, rate=0.1):
        super(custom_CNN, self).__init__()
        self.model = models.resnet50(pretrained=False)
        self.model.fc.out_features=1
        self.batchnorm = nn.BatchNorm1d(1000)
        self.relu =nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(1000,1,bias=True)
    
    def forward(self, inputs):
        output = self.linear((self.model(inputs)))
        return output


In [ ]:
model = custom_CNN().cuda()

In [38]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

In [39]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.cuda()
            label = label.to(self.device).float()
            pred = torch.sigmoid(self.model(img)).view(-1)
            pred_value = (pred>0.5).int()
            loss = self.loss_fn(pred, label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred_value.cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred_value.cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = torch.sigmoid(self.model(img)).view(-1)
            
            loss = self.loss_fn(pred, label)
            pred_value = (pred>=0.5).int()
            val_total_loss += loss.item()
            prob_lst.extend(pred_value.cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred_value.cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

In [40]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

In [41]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=WORKING_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=WORKING_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


In [109]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001,
    epochs=EPOCHS, steps_per_epoch=len(train_dataloader)
)
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn
trainer = Trainer(loss_fn, model_Alex, DEVICE, metric_fn, optimizer, scheduler)
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [110]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)
44, Acc: 0.5076923076923077, F1-Macro: 0.336734693877551
Epoch 1, Train loss: 0.732153637541665, Acc: 0.4629948364888124, F1-Macro: 0.3662527269676121
  4% 2/50 [01:29<35:32, 44.43s/it]
    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, '/USER/Taeyun/best.pt')


  0% 0/50 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7321115434169769, Acc: 0.4750430292598967, F1-Macro: 0.38717531064009325


  2% 1/50 [00:44<36:07, 44.23s/it]

Epoch 0, Val loss: 1.0367943942546844, Acc: 0.5076923076923077, F1-Macro: 0.336734693877551
Epoch 1, Train loss: 0.732153637541665, Acc: 0.4629948364888124, F1-Macro: 0.3662527269676121


  4% 2/50 [01:29<35:32, 44.43s/it]

...

Epoch 3, Val loss: 1.0431052446365356, Acc: 0.5076923076923077, F1-Macro: 0.336734693877551
Early stopping counter 3/10
Epoch 4, Train loss: 0.7324527237150404, Acc: 0.45438898450946646, F1-Macro: 0.36498791533494


 10% 5/50 [03:43<33:34, 44.76s/it]

Epoch 4, Val loss: 1.042238861322403, Acc: 0.5076923076923077, F1-Macro: 0.336734693877551
Early stopping counter 4/10


 10% 5/50 [04:03<36:27, 48.60s/it]


KeyboardInterrupt: 

In [38]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [40]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=WORKING_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


In [43]:
model.load_state_dict(torch.load('/USER/Taeyun/best.pt')['model'])
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = torch.sigmoid(model(img)).view(-1)
        pred_value =(pred>=0.5).int()
        file_lst.extend(list(file_num))
        pred_lst.extend(pred_value.tolist())
        prob_lst.extend(pred.tolist())

4it [00:07,  1.75s/it]


In [44]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction.csv', index=False)